In [1]:
setwd("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1005. Miscellaneous Analyses/P1005.4. Clinical Sample EVs/P1005.4.2. Analyses/Input")
Output = ("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1005. Miscellaneous Analyses/P1005.4. Clinical Sample EVs/P1005.4.2. Analyses/Output")

cur_date = "121522"

library(tidyverse)
library(pdftools)
library(tm)

# only grabs files ending in "pdf"
pdf_list = list.files(pattern = "pdf$")
head(pdf_list)    

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Using poppler version 22.02.0

Loading required package: NLP


Attaching package: ‘NLP’


The following object is masked from ‘package:ggplot2’:

    annotate




[1] "20221213_0004_18F6_80_size_488.pdf" "20221213_0005_18F6_90_size_488.pdf"
[3] "20221213_0006_18F6_70_size_488.pdf" "20221213_0007_18F7_70_size_488.pdf"
[5] "20221213_0008_18F7_80_size_488.pdf" "20221213_0009_18F7_89_size_488.pdf"

# Data we want to extract:
1. Pdf file name as the sample identifier
2. Original Concentration
3. Some of the ‘X Values’ table listed on the bottom left, specifically the: X10 Concentration, X50 Concentration, X90 Concentration, and Mean Concentration, and StdDev Concentration

In [2]:
# this code I pulled online and I'm not entirely sure how each line of code works, but it just parses the pdf files
corpus_raw <- data.frame("Sample Identifier" = c(), "Text" = c())

for (i in 1:length(pdf_list)){
    document_text = pdf_text(paste(pdf_list[i],sep = "")) %>% 
    strsplit("\n")
    
        document = data.frame("Sample Identifier" = gsub(x = pdf_list[i], pattern = ".pdf", replacement = ""), 
        "Text" = document_text, stringsAsFactors = FALSE)
    
    colnames(document) <- c("Sample Identifier", "Text")
    corpus_raw <- rbind(corpus_raw,document) 
}

# still haven't figured out why there are duplicates
corpus_raw = unique(corpus_raw)

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: I

In [3]:
# viewing data
# dataframe contains a sample identifier column and a text column
# sample identifier column: name of pdf file
# text column: includes data/ text from the pdf
head(corpus_raw)

,Sample Identifier,Text
,<chr>,<chr>
1,20221213_0004_18F6_80_size_488,Electrophoresis & Brownian Motion
2,20221213_0004_18F6_80_size_488,Video Analysis
3,20221213_0004_18F6_80_size_488,Laser Scattering Microscopy
4,20221213_0004_18F6_80_size_488,www.particle-metrix.de
5,20221213_0004_18F6_80_size_488,
6,20221213_0004_18F6_80_size_488,Operator (Report): sop_kabanovlab.inst


In [4]:
# getting the original concentration from the text column and creating a df
OG_concentration_df = corpus_raw %>%
    filter(grepl("Original Concentration", Text)) %>%
    separate(Text, c(NA, "Original Concentration"), sep = ": ") %>%
    separate(`Original Concentration`, c("Original Concentration (Particles/ mL)", NA), sep = "P")
head(OG_concentration_df)

,Sample Identifier,Original Concentration (Particles/ mL)
,<chr>,<chr>
1,20221213_0004_18F6_80_size_488,3.1E+11
2,20221213_0005_18F6_90_size_488,1.1E+12
3,20221213_0006_18F6_70_size_488,1.3E+11
4,20221213_0007_18F7_70_size_488,4.4E+10
5,20221213_0008_18F7_80_size_488,1.9E+11
6,20221213_0009_18F7_89_size_488,7.3E+11


In [5]:
# doing the same with the concentration data
X_concentration_df = corpus_raw %>%
    filter(grepl("X10", Text)| grepl("X50 ", Text)| grepl("X90", Text) | grepl("Mean", Text)| grepl("StdDev", Text)) %>%
    separate(Text, c("Text", NA), sep = "                ") %>%
    separate(Text, c("Text", "Concentration"), sep = "       ") %>%
    pivot_wider(names_from = Text, values_from = Concentration)
head(X_concentration_df)

Sample Identifier,X10,X50,X90,Mean,StdDev
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
20221213_0004_18F6_80_size_488,82.5,121.7,183.8,131.5,39.9
20221213_0005_18F6_90_size_488,40.2,66.6,108.4,75.2,29.9
20221213_0006_18F6_70_size_488,99.4,155.6,202.1,156.2,42.5
20221213_0007_18F7_70_size_488,89.2,130.6,196.4,139.7,42.7
20221213_0008_18F7_80_size_488,75.9,106.0,149.2,113.9,33.7
20221213_0009_18F7_89_size_488,44.7,70.1,106.3,76.8,25.5


In [6]:
# combining into one final df
final_df = full_join(OG_concentration_df, X_concentration_df)
head(final_df)

Joining, by = "Sample Identifier"


,Sample Identifier,Original Concentration (Particles/ mL),X10,X50,X90,Mean,StdDev
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,20221213_0004_18F6_80_size_488,3.1E+11,82.5,121.7,183.8,131.5,39.9
2,20221213_0005_18F6_90_size_488,1.1E+12,40.2,66.6,108.4,75.2,29.9
3,20221213_0006_18F6_70_size_488,1.3E+11,99.4,155.6,202.1,156.2,42.5
4,20221213_0007_18F7_70_size_488,4.4E+10,89.2,130.6,196.4,139.7,42.7
5,20221213_0008_18F7_80_size_488,1.9E+11,75.9,106.0,149.2,113.9,33.7
6,20221213_0009_18F7_89_size_488,7.3E+11,44.7,70.1,106.3,76.8,25.5


In [7]:
# exporting data
write.csv(final_df, paste0(Output,"/", cur_date, "_Final_PDF_Data.csv"), row.names = FALSE)